# DataLoader

In [165]:
import pandas as pd
import numpy as np

import os

In [166]:
data_dir = "../../kaggle_data/creditcard_overdue/"
Raw_data_dir = os.path.join(data_dir, "open/")
pre_data_dir = os.path.join(data_dir, "preprocess/")

Raw_train = pd.read_csv(os.path.join(Raw_data_dir, "train.csv"))
Raw_test = pd.read_csv(os.path.join(Raw_data_dir, "test.csv"))

pre_train = pd.read_csv(os.path.join(pre_data_dir, "train_preprocess.csv"))
pre_test = pd.read_csv(os.path.join(pre_data_dir, "test_preprocess.csv"))

submission = pd.read_csv(os.path.join(Raw_data_dir, "sample_submission.csv"))

In [167]:
Raw_train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


# Preprocessing

## 불필요한 Feature 제거

In [168]:
# Index, FLAG_MOBIL 제거
train = Raw_train.drop(['index', 'FLAG_MOBIL'], axis=1)
test = Raw_test.drop(['index', 'FLAG_MOBIL'], axis=1)

## Occyp_type

- 많이 출현한 값부터 차례로 가장 높은 숫자 부여 or LebelEncoder
    - case가 19가지 이므로 0~18까지 feature값이 크게 차이남.
- One hot Encoding 하는 것이 가장 좋아 보임.

In [169]:
train['occyp_type'].unique()

array([nan, 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'Realty agents', 'Security staff', 'Cleaning staff',
       'Private service staff', 'Cooking staff', 'Secretaries',
       'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'], dtype=object)

In [170]:
train['occyp_type'].value_counts()

Laborers                 4512
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1575
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            403
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      124
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64

In [171]:
print(f"nan rate : {np.sum(train['occyp_type'].isnull()) / len(train) * 100}")

nan rate : 30.884076047926822


In [172]:
train['occyp_type'] = train['occyp_type'].fillna("Unknown")
print(np.sum(train['occyp_type'].isnull()))

0


In [173]:
occyp = train['occyp_type'].value_counts()

print(occyp)

occyp = pd.DataFrame(occyp)

occyp_list = list(occyp.index)

occyp_list

Unknown                  8171
Laborers                 4512
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1575
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            403
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      124
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64


['Unknown',
 'Laborers',
 'Core staff',
 'Sales staff',
 'Managers',
 'Drivers',
 'High skill tech staff',
 'Accountants',
 'Medicine staff',
 'Cooking staff',
 'Security staff',
 'Cleaning staff',
 'Private service staff',
 'Low-skill Laborers',
 'Waiters/barmen staff',
 'Secretaries',
 'Realty agents',
 'HR staff',
 'IT staff']

In [174]:
train = pd.get_dummies(train, columns=['occyp_type'])

In [181]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Unknown,occyp_type_Waiters/barmen staff
0,0,0,0,0,202500.0,Commercial associate,3,Married,Municipal apartment,-13899,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,1,247500.0,Commercial associate,2,Civil marriage,House / apartment,-11380,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,450000.0,Working,3,Married,House / apartment,-19087,...,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,202500.0,Commercial associate,2,Married,House / apartment,-15088,...,0,0,0,0,0,1,0,0,0,0
4,0,1,1,0,157500.0,State servant,3,Married,House / apartment,-15037,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,State servant,2,Married,House / apartment,-12079,...,0,0,0,0,0,0,0,0,0,0
26453,0,0,1,1,180000.0,Working,3,Separated,House / apartment,-15291,...,0,0,0,0,0,0,0,0,1,0
26454,0,1,0,0,292500.0,Working,2,Civil marriage,With parents,-10082,...,0,0,0,0,0,0,0,0,0,0
26455,1,0,1,0,171000.0,Working,1,Single / not married,House / apartment,-10145,...,0,0,0,0,0,0,0,0,0,0


## 범주형 Feature 실수화

In [175]:
train['gender'] = train['gender'].replace(['F', 'M'], [0, 1])
test['gender'] = test['gender'].replace(['F', 'M'], [0, 1])
print('gender : ')
print(train['gender'].value_counts())
print("----------------------------")

train['car'] = train['car'].replace(['N', 'Y'], [0, 1])
test['car'] = test['car'].replace(['N', 'Y'], [0, 1])
print('car : ')
print(train['car'].value_counts())
print("----------------------------")

train['reality'] = train['reality'].replace(['N', 'Y'], [0, 1])
test['reality'] = test['reality'].replace(['N', 'Y'], [0, 1])
print('reality : ')
print(train['reality'].value_counts())
print("----------------------------")

train['edu_type'] = train['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])
test['edu_type'] = test['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])
print('edu_type : ')
print(train['edu_type'].value_counts())
print("----------------------------")

gender : 
0    17697
1     8760
Name: gender, dtype: int64
----------------------------
car : 
0    16410
1    10047
Name: car, dtype: int64
----------------------------
reality : 
1    17830
0     8627
Name: reality, dtype: int64
----------------------------
edu_type : 
2    17995
3     7162
1     1020
0      257
4       23
Name: edu_type, dtype: int64
----------------------------


## Income total

In [176]:
# income total 전처리

import matplotlib.pyplot as plt
import seaborn as sns

In [177]:
income_counts = train['income_total'].value_counts()
unique_income = income_counts.index

income_counts = pd.DataFrame(income_counts)
income_counts.index = range(len(income_counts))
income_counts['income_value'] = unique_income
income_counts.columns = ['counts', 'income_values']

income_counts

,counts,income_values
0,3164,135000.0
1,2233,157500.0
2,2225,180000.0
3,2178,112500.0
4,2170,225000.0
...,...,...
244,1,227250.0
245,1,91530.0
246,1,39600.0
247,1,432000.0


In [178]:
max_income = income_counts['income_values'].max()
min_income = income_counts['income_values'].min()

print(max_income, min_income)

1575000.0 27000.0


In [179]:
num_bins = [5, 7, 10, 14]

for num_bins_ in num_bins:
    train_ = train.copy()
    test_ = test.copy()

    train_['income_total'] = train_['income_total']/10000
    test_['income_total'] = test_['income_total']/10000

    count, bin_dividers =np.histogram(train_['income_total'],bins=num_bins_)
    bin_names=['소득'+str(i) for i in range(num_bins_) ]

    #bin_dividers는 train기준!!
    train_['income_total']=pd.cut(x=train_['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)
    test_['income_total']=pd.cut(x=test_['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)

    print(f"========[num_bins : {num_bins_}]==========")
    print("train bin counts : ")
    print(train_['income_total'].value_counts())

    train_.to_csv(f"{num_bins_}_bins_train.csv", index=False)
    print("=================================")


========[num_bins : 5]==========
train bin counts : 
소득0    24542
소득1     1736
소득2      165
소득4        9
소득3        5
Name: income_total, dtype: int64
========[num_bins : 7]==========
train bin counts : 
소득0    21601
소득1     4462
소득2      305
소득3       71
소득4        9
소득6        5
소득5        4
Name: income_total, dtype: int64
========[num_bins : 10]==========
train bin counts : 
소득0    16212
소득1     8330
소득2     1530
소득3      206
소득4      121
소득5       44
소득9        5
소득8        4
소득6        3
소득7        2
Name: income_total, dtype: int64
========[num_bins : 14]==========
train bin counts : 
소득1     11123
소득0     10478
소득2      3187
소득3      1275
소득4       157
소득5       148
소득7        40
소득6        31
소득8         7
소득13        5
소득11        4
소득9         2
소득10        0
소득12        0
Name: income_total, dtype: int64


In [182]:
bin_5 = pd.read_csv("5_bins_train.csv")

bin_5

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Unknown,occyp_type_Waiters/barmen staff
0,0,0,0,0,소득0,Commercial associate,3,Married,Municipal apartment,-13899,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,1,소득0,Commercial associate,2,Civil marriage,House / apartment,-11380,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,소득1,Working,3,Married,House / apartment,-19087,...,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,소득0,Commercial associate,2,Married,House / apartment,-15088,...,0,0,0,0,0,1,0,0,0,0
4,0,1,1,0,소득0,State servant,3,Married,House / apartment,-15037,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,소득0,State servant,2,Married,House / apartment,-12079,...,0,0,0,0,0,0,0,0,0,0
26453,0,0,1,1,소득0,Working,3,Separated,House / apartment,-15291,...,0,0,0,0,0,0,0,0,1,0
26454,0,1,0,0,소득0,Working,2,Civil marriage,With parents,-10082,...,0,0,0,0,0,0,0,0,0,0
26455,1,0,1,0,소득0,Working,1,Single / not married,House / apartment,-10145,...,0,0,0,0,0,0,0,0,0,0
